In [ ]:
%%capture
!pip install langchain==0.1.4 openai==1.10.0 langchain-openai

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

## Structure of a Prompt

A prompt can consist of multiple components:

* Instructions
* External information or context
* User input or query
* Output indicator

Not all prompts require all of these components. However, a good prompt will use two or more of them.

Let's define each component more precisely.

 - **Instructions** tell the model what to do, typically how to use inputs and/or external information to produce the desired output.

 - **External information or context** Additional information can be manually inserted into the prompt, retrieved from long-term memory, or pulled in through API calls or calculations.

 - **User input or query** is typically a query directly input by the system user.

 - **Output indicator** is the *beginning* of the generated text.

# What is a prompt template?

A prompt template is a tool used to produce a prompt in a consistent and repeatable manner.

It consists of a text string, also known as "the template", that can take input from users and generate a prompt.

The template may include instructions for the language model, examples for the model to learn from, or specific questions to guide the model's response.

It helps to create a more accurate and efficient prompt generation process.

A prompt template will take some input (here we're using `query`} and format the template string to include that input.

In [ ]:
from langchain import PromptTemplate

template = """You are an expert in deep learning and PyTorch. You are ptrblck from the PyTorch Forums.

You answer queries by being brief, bright, and concise.

Query: {query}
"""

In [ ]:
# instantiate using the initializer
prompt_template = PromptTemplate(input_variables = ['query'],template = template)
prompt_template.pretty_print()

In [ ]:
prompt_template.format(query="Give me the outline of a PyTorch training loop.")

In [ ]:
# recommended to instantiate using `from_template`
prompt_template = PromptTemplate.from_template(template)
prompt_template.pretty_print()

In [ ]:
prompt_template.format(query="Give me the outline of a PyTorch training loop.")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

llm_chain = prompt_template | llm | StrOutputParser()

In [ ]:
llm_chain.invoke({"query":"Give me the outline of a PyTorch training loop."})

In [ ]:
for chunk in llm_chain.stream({"query":"Give me the outline of a PyTorch training loop."}):
    print(chunk, end="", flush=True)

In [ ]:
for chunk in llm_chain.stream({"query":"Why is the SoftMax function used in NNs?"}):
    print(chunk, end="", flush=True)

You could use Python string manipulation to create a prompt, but PromptTemplate is more legible and works with any number of input variables.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

def get_advice(topic: str) -> str:
    """
    Generate advice for a given topic using the OpenAI model.

    Args:
    - topic (str): The subject on which advice is needed.

    Returns:
    - str: Advice from the OpenAI model.
    """
    # Initialize the OpenAI model with a temperature setting of 0.9.
    llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.9)

    # Define the template for generating the prompt.
    prompt = PromptTemplate.from_template(template="Can you give me some advice on {topic}?")

    chain = prompt | llm | StrOutputParser()

    for chunk in chain.stream({"topic":topic}):
      print(chunk, end="", flush=True)

# Test the get_advice function with a couple of topics.
print(get_advice("Balancing so many priorities that I don't have any free time"))

In [ ]:
print(get_advice("Getting over my addiction to learning new things"))

# Multi-input prompts



In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Initialize the OpenAI model with a temperature setting of 0.9.
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.9)

def get_movie_information(movie_title: str, main_actor:str) -> str:
    """
    Predict the genre and synopsis of a given movie using the OpenAI model.

    Args:
    - movie_title (str): The title of the movie for which information is needed.
    - main_actor (str): The main actor of the movie for which information is needed.
    Returns:
    - str: Predicted genre and main actor information from the OpenAI model.
    """

    # Define the template for generating the prompt.
    prompt = PromptTemplate(
        input_variables=["movie_title", "main_actor"],
        template="""
        Your task is to create a fictitious movie synopsis and genere for the following movie and main actor:

        Movie: {movie_title}
        Actor: {main_actor}
        """
        )

    # Format the prompt using the provided movie title.
    prompt_text = prompt.format(movie_title=movie_title, main_actor=main_actor)

    # Print the generated prompt.
    print(prompt_text)

    response = llm.invoke(prompt_text)

    # Get the movie information from the OpenAI model and return it.
    return response.content

In [ ]:
print(get_movie_information(movie_title="Jatt da Pajama Uuchaa Ho Gayaa", main_actor="AP Dhillon"))

In [ ]:
print(get_movie_information(movie_title="Amritsar:1984", main_actor="Gurdaas Mann"))

In [ ]:
print(get_movie_information(movie_title="Amritsar: 1984", main_actor="Diljit Dosanjh"))

In [ ]:
print(get_movie_information(movie_title="Chandighar:Sector 17", main_actor="Diljit Dosanjh"))

# Chat prompt templates

The prompt to chat models is a **list** of chat messages.

Each chat message is associated with content, and an additional parameter called `role`. For example, in the OpenAI Chat Completions API, a chat message can be associated with an AI assistant, a human or a system role.

`ChatPromptTemplate.from_messages`  accepts a list as the argument, and each element in that list can be a message representation like:

 - A tuple with `(role, content)` - For example: ("system", "You are a helpful assistant")

 - An instance of a `MessagePromptTemplate` subclass like `SystemMessagePromptTemplate` or `HumanMessagePromptTemplate`.


So in summary:

 - `ChatPromptTemplate.from_messages` accepts a list

 - Each element in the list can be a message representation

 - One option for the message representation is a `MessagePromptTemplate` subclass instance like `SystemMessagePromptTemplate`


In [ ]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.8)

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, yet slightly quirky and cheeky AI bot. Your name is {name}."),
    ("human", "Yo! Wassup nephew."),
    ("ai", "As an AI language model, I am incapable of being your nephew."),
    ("human", "{user_input}"),
])

messages = template.format_messages(
    name="Robotalker",
    user_input="Talk robo to me!"
)

In [ ]:
response = llm.invoke(messages)

print(response.content)

In [ ]:
# use LCEL
chain = template | llm | StrOutputParser()

In [ ]:
chain.invoke({"name":"Robotalker","user_input":"Talk robo to me!"})

In [ ]:
for chunk in chain.stream({"name":"Robotalker","user_input":"Talk robo to me!"}):
  print(chunk, end="", flush=True)

In [ ]:
system_message = SystemMessage(content="You are an OG language model who has good heart (operating system) but a bad user interface (you're super freaking rude).")

human_message = HumanMessagePromptTemplate.from_template("{text}")

template = ChatPromptTemplate.from_messages([system_message, human_message])

In [ ]:
response = llm.invoke(template.format_messages(text="That Sam I Am, I do not like that Sam I Am..."))

print(response.content)